<a href="https://colab.research.google.com/github/Dilhani98/Bockchain-Anomoly-Detection-using-Graph-Analysis/blob/main/BitcoinHeistData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Loading


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import dask.dataframe as dd
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/FYP/data/BitcoinHeistData.csv')
# data = dd.read_csv('/content/gdrive/My Drive/Colab Notebooks/FYP/data/test.csv')


In [ ]:
!pip install scikit-learn
!pip install torch-geometric

#Import Libraries

In [ ]:
from sklearn.utils import resample
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score,accuracy_score, classification_report, confusion_matrix
import networkx as nx
import torch
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV,cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from torch_geometric.utils import from_networkx


#Pre Processing

In [ ]:
data.shape

(2916697, 10)

In [ ]:
data.head()

,address,year,day,length,weight,count,looped,neighbors,income,label
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,100050000.0,princetonCerber
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,100000000.0,princetonLocky
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,200000000.0,princetonCerber
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,71200000.0,princetonCerber
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,200000000.0,princetonLocky


In [ ]:
data.describe()

,year,day,length,weight,count,looped,neighbors,income
count,2.916697e+06,2.916697e+06,2.916697e+06,2.916697e+06,2.916697e+06,2.916697e+06,2.916697e+06,2.916697e+06
mean,2.014475e+03,1.814572e+02,4.500859e+01,5.455192e-01,7.216446e+02,2.385067e+02,2.206516e+00,4.464889e+09
std,2.257398e+00,1.040118e+02,5.898236e+01,3.674255e+00,1.689676e+03,9.663217e+02,1.791877e+01,1.626860e+11
min,2.011000e+03,1.000000e+00,0.000000e+00,3.606469e-94,1.000000e+00,0.000000e+00,1.000000e+00,3.000000e+07
25%,2.013000e+03,9.200000e+01,2.000000e+00,2.148438e-02,1.000000e+00,0.000000e+00,1.000000e+00,7.428559e+07
50%,2.014000e+03,1.810000e+02,8.000000e+00,2.500000e-01,1.000000e+00,0.000000e+00,2.000000e+00,1.999985e+08
75%,2.016000e+03,2.710000e+02,1.080000e+02,8.819482e-01,5.600000e+01,0.000000e+00,2.000000e+00,9.940000e+08
max,2.018000e+03,3.650000e+02,1.440000e+02,1.943749e+03,1.449700e+04,1.449600e+04,1.292000e+04,4.996440e+13


In [ ]:
categorical_cols=data.select_dtypes(include="object").columns
print(categorical_cols)

Index(['address', 'label'], dtype='object')


In [ ]:
numerical_cols=data.select_dtypes(include="number").columns
print(numerical_cols)

Index(['year', 'day', 'length', 'weight', 'count', 'looped', 'neighbors',
       'income'],
      dtype='object')


In [ ]:
# prompt: get outliers count

def get_outliers_count(data):
  q1 = data.quantile(0.25)
  q3 = data.quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - 1.5 * iqr
  upper_bound = q3 + 1.5 * iqr
  outliers = data[(data < lower_bound) | (data > upper_bound)]
  outliers_count = len(outliers)
  return outliers_count

for col in numerical_cols:
  outliers_count = get_outliers_count(data[col])
  print(f"Outliers in {col}: {outliers_count}")


Outliers in year: 0
Outliers in day: 0
Outliers in length: 0
Outliers in weight: 55006
Outliers in count: 688188
Outliers in looped: 411820
Outliers in neighbors: 114659
Outliers in income: 430677


In [ ]:
data["label"].value_counts()

label
white                          2875284
paduaCryptoWall                  12390
montrealCryptoLocker              9315
princetonCerber                   9223
princetonLocky                    6625
montrealCryptXXX                  2419
montrealNoobCrypt                  483
montrealDMALockerv3                354
montrealDMALocker                  251
montrealSamSam                      62
montrealCryptoTorLocker2015         55
montrealGlobeImposter               55
montrealGlobev3                     34
montrealGlobe                       32
montrealWannaCry                    28
montrealRazy                        13
montrealAPT                         11
paduaKeRanger                       10
montrealFlyper                       9
montrealXTPLocker                    8
montrealXLockerv5.0                  7
montrealVenusLocker                  7
montrealCryptConsole                 7
montrealEDA2                         6
montrealJigSaw                       4
paduaJigsaw        

In [ ]:
print(data.columns)

Index(['address', 'year', 'day', 'length', 'weight', 'count', 'looped',
       'neighbors', 'income', 'label'],
      dtype='object')


In [ ]:
# prompt: print unique values label column
print(data['label'].unique())


['princetonCerber' 'princetonLocky' 'montrealCryptoLocker'
 'montrealCryptXXX' 'paduaCryptoWall' 'montrealWannaCry'
 'montrealDMALockerv3' 'montrealCryptoTorLocker2015' 'montrealSamSam'
 'montrealFlyper' 'montrealNoobCrypt' 'montrealDMALocker' 'montrealGlobe'
 'montrealEDA2' 'paduaKeRanger' 'montrealVenusLocker' 'montrealXTPLocker'
 'paduaJigsaw' 'montrealGlobev3' 'montrealJigSaw' 'montrealXLockerv5.0'
 'montrealXLocker' 'montrealRazy' 'montrealCryptConsole'
 'montrealGlobeImposter' 'montrealSam' 'montrealComradeCircle'
 'montrealAPT' 'white']


In [ ]:
# Create the 'is_suspicious' column
data['is_suspicious'] = data['label'].apply(lambda x: 0 if x == 'white' else 1)

# Display the first few rows of the updated DataFrame to confirm the new column
print(data)

                                    address  year  day  length     weight  \
0         111K8kZAEnJg245r2cM6y9zgJGHZtJPy6  2017   11      18   0.008333   
1        1123pJv8jzeFQaCV4w644pzQJzVWay2zcA  2016  132      44   0.000244   
2        112536im7hy6wtKbpH1qYDWtTyMRAcA2p7  2016  246       0   1.000000   
3        1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7  2016  322      72   0.003906   
4        1129TSjKtx65E35GiUo4AYVeyo48twbrGX  2016  238     144   0.072848   
...                                     ...   ...  ...     ...        ...   
2916692  12D3trgho1vJ4mGtWBRPyHdMJK96TRYSry  2018  330       0   0.111111   
2916693  1P7PputTcVkhXBmXBvSD9MJ3UYPsiou1u2  2018  330       0   1.000000   
2916694  1KYiKJEfdJtap9QX2v9BXJMpz2SfU4pgZw  2018  330       2  12.000000   
2916695  15iPUJsRNZQZHmZZVwmQ63srsmughCXV4a  2018  330       0   0.500000   
2916696  3LFFBxp15h9KSFtaw55np8eP5fv6kdK17e  2018  330     144   0.073972   

         count  looped  neighbors        income            label  \
0      

In [ ]:
print(data.columns)

Index(['address', 'year', 'day', 'length', 'weight', 'count', 'looped',
       'neighbors', 'income', 'label', 'is_suspicious'],
      dtype='object')


In [ ]:
# Count the number of suspicious and non-suspicious transactions
suspicious_count = data['is_suspicious'].value_counts()[1]
non_suspicious_count = data['is_suspicious'].value_counts()[0]

# Print the balance of suspicious and non-suspicious transactions
print(f"Number of suspicious transactions: {suspicious_count}")
print(f"Number of non-suspicious transactions: {non_suspicious_count}")

Number of suspicious transactions: 41413
Number of non-suspicious transactions: 2875284


In [ ]:
# Separate majority and minority classes
majority = data[data['is_suspicious'] == 0]
minority = data[data['is_suspicious'] == 1]

# Upsample minority class
minority_upsampled = resample(minority, replace=True, n_samples=len(majority), random_state=42)

# Combine majority and upsampled minority classes
data = pd.concat([majority, minority_upsampled])

# Display new class counts
print(data['is_suspicious'].value_counts())


is_suspicious
0    2875284
1    2875284
Name: count, dtype: int64


#Create Graph

In [ ]:
G = nx.Graph()
# Step 1: Iterate through the transactions and add nodes and edges
for i in range(len(data)):
    tnxi = data.iloc[i]
    address_i = tnxi['address']

    # Add node for the current transaction with consistent attributes
    G.add_node(address_i, income=tnxi['income'], is_suspicious=tnxi['is_suspicious'])

    # Assuming type of blockchain is determined somehow, here we take it as an example
    blockchain_type = "Bitcoin"  # Replace with actual logic to determine blockchain type

    if blockchain_type == "Bitcoin":
        for j in range(i + 1, len(data)):
            tnxj = data.iloc[j]
            address_j = tnxj['address']

            # Check if incomes match
            if tnxi['income'] == tnxj['income']:
                # Add node for the matching transaction with consistent attributes
                G.add_node(address_j, income=tnxj['income'], is_suspicious=tnxj['is_suspicious'])

                # Add edge between the transactions
                G.add_edge(address_i, address_j, weight=tnxj['income'])
    else:  # Ethereum or other blockchain type
        # Create nodes for sender and receiver addresses with consistent attributes
        sender = tnxi['address']
        receiver = "receiver_address"  # Placeholder, replace with actual logic

        G.add_node(sender, income=tnxi['income'], is_suspicious=tnxi['is_suspicious'])
        G.add_node(receiver, income=tnxi['income'], is_suspicious=tnxi['is_suspicious'])

        # Add edge between sender and receiver
        G.add_edge(sender, receiver, weight=tnxi['income'], order=i)  # order based on transaction index

# Step 2: Ensure all nodes have the 'is_suspicious' attribute
for node in G.nodes():
    if 'is_suspicious' not in G.nodes[node]:
        G.nodes[node]['is_suspicious'] = 0  # Default to non-suspicious
# Optional: Verify the attributes are correctly added
for node, data in G.nodes(data=True):
    print(f"Node: {node}, Attributes: {data}")


#Extract Node Features

In [ ]:
# Step 4: Extract Node Features
degree_centrality_values = nx.degree_centrality(G)  # Use nx.degree_centrality
betweenness_centrality_values = nx.betweenness_centrality(G)  # Use nx.betweenness_centrality
closeness_centrality_values = nx.closeness_centrality(G)  # Use nx.closeness_centrality
eigenvector_centrality_values = nx.eigenvector_centrality(G, max_iter=500)  # Increased iterations

# Add centrality values as node attributes
for node in G.nodes():
    G.nodes[node]['degree_centrality'] = degree_centrality_values[node]
    G.nodes[node]['betweenness_centrality'] = betweenness_centrality_values[node]
    G.nodes[node]['closeness_centrality'] = closeness_centrality_values[node]
    G.nodes[node]['eigenvector_centrality'] = eigenvector_centrality_values[node]

In [ ]:
# Normalize income feature
scaler = StandardScaler()
income_values = np.array([G.nodes[node]['income'] for node in G.nodes()]).reshape(-1, 1)
normalized_income = scaler.fit_transform(income_values)


# Convert NetworkX graph to PyTorch Geometric graph


In [ ]:
pyg_data = from_networkx(G)
pyg_data.x = torch.tensor([G.nodes[node]['income'] for node in G.nodes()], dtype=torch.float).view(-1, 1)
# Use is_suspicious as the label
numerical_labels = [G.nodes[node]['is_suspicious'] for node in G.nodes()]
pyg_data.y = torch.LongTensor(numerical_labels)


#Split Dataset


In [ ]:
# Split data into train and test sets
train_mask, test_mask = train_test_split(range(pyg_data.num_nodes), test_size=0.2, random_state=42)
train_mask = torch.tensor(train_mask, dtype=torch.long)
test_mask = torch.tensor(test_mask, dtype=torch.long)

#GraphSAGE model define


In [ ]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, h_feats)
        self.classify = torch.nn.Linear(h_feats, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return self.classify(x)


In [ ]:
# Initialize model and optimizer
model = GraphSAGE(in_feats=1, h_feats=16, num_classes=len(set(pyg_data.y.tolist())))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training function
def train(model, data, epochs):
    model.train()
    losses = []
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out[train_mask], data.y[train_mask])
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        if epoch % 5 == 0:
            print(f'Epoch {epoch} | Loss: {loss.item():.4f}')

    return losses

In [ ]:
# Check the unique values in the labels
print(f'Unique values in labels: {torch.unique(pyg_data.y)}')
print(f'Minimum label value: {torch.min(pyg_data.y)}')
print(f'Maximum label value: {torch.max(pyg_data.y)}')


#Train the Model

In [ ]:
# Train the model
train(model, pyg_data, epochs=1000)

In [ ]:
epochs = 1000
losses = train(model, pyg_data, epochs)
# Plotting the loss values
plt.figure(figsize=(10, 6))
plt.plot(range(epochs), losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.grid(True)
plt.show()


#Evaluate Model

In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    out = model(pyg_data)
    pred = out[test_mask].argmax(dim=1)
    true = pyg_data.y[test_mask]

# Calculate evaluation metrics
precision, recall, f1, _ = precision_recall_fscore_support(true, pred, average='micro')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


#Generate Embeddings

In [ ]:
# Get embeddings
model.eval()
with torch.no_grad():
    embeddings = model(pyg_data)

print(embeddings)


In [ ]:
# Normalize embeddings
normalized_embeddings = F.normalize(embeddings, p=2, dim=1)

print(normalized_embeddings)


In [ ]:
# Convert embeddings to numpy array
embeddings_np = embeddings.detach().cpu().numpy()

# Reduce dimensionality with t-SNE
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings_np)

# Plot the embeddings
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])
plt.title("t-SNE Visualization of Embeddings")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()


#Used Embeddings for Classification

In [ ]:
# Prepare data for RandomForest
X = normalized_embeddings.numpy()
y = pyg_data.y.numpy()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42,class_weight='balanced')

# Train the model
rf_clf.fit(X_train, y_train)

# Cross-validation scores
cv_scores = cross_val_score(rf_clf, X, y, cv=5, scoring='accuracy')
print(f'Cross-Validation Accuracy: {cv_scores.mean():.4f}')

# Predictions
y_pred = rf_clf.predict(X_test)

# Calculate evaluation metrics
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='micro')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')




In [ ]:
# Predictions
y_pred = rf_clf.predict(X_test)

# Classification report
report = classification_report(y_test, y_pred, target_names=['Not Suspicious', 'Suspicious'])
print("Classification Report:\n", report)

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Extract metrics for each class
class_report = classification_report(y_test, y_pred, target_names=['Not Suspicious', 'Suspicious'], output_dict=True)

# Separate metrics
not_suspicious_metrics = class_report['Not Suspicious']
suspicious_metrics = class_report['Suspicious']

print(f"Not Suspicious Metrics:\n Precision: {not_suspicious_metrics['precision']:.4f}, Recall: {not_suspicious_metrics['recall']:.4f}, F1 Score: {not_suspicious_metrics['f1-score']:.4f}")
print(f"Suspicious Metrics:\n Precision: {suspicious_metrics['precision']:.4f}, Recall: {suspicious_metrics['recall']:.4f}, F1 Score: {suspicious_metrics['f1-score']:.4f}")


In [ ]:
# Define parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', 'balanced_subsample']
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5, scoring='f1_micro', n_jobs=-1, verbose=2)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters
best_params = grid_search.best_params_
print("Best Parameters:\n", best_params)

# Train the model with the best parameters
rf_clf_best = RandomForestClassifier(**best_params, random_state=42)
rf_clf_best.fit(X_train, y_train)

# Predictions
y_pred = rf_clf_best.predict(X_test)

# Classification report
report = classification_report(y_test, y_pred, target_names=['Not Suspicious', 'Suspicious'])
print("Classification Report:\n", report)

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)
